In [1]:
import re
import numpy as np

from itertools import chain

In [2]:
import email
from email.utils import parseaddr


In [3]:
import pandas as pd

Задание 1 (10 баллов)

Прочитайте данные из emails.csv. В колонке message содержится информация о письме. Вам нужно сформировать несколько таблиц по данным для комфортной работы.

Во-первых, у Вас имеется информация от том от когдо было отправленно письмо и кому:

From - email отправителя
X-From - имя отправитела
To - email получателя
X-To - имя получателя
Каждому индивидууму нужно присвоить свой уникальный id и сформировать таблицу, где по нему можно получить email и имя.

Также Вам нужно сформировать таблицу, в которой по каждому письму будет содержаться:

Дата
id отправителя
id получателей (обратитие внимание на то, что одно письмо может получать несколько человек)
Текст

In [5]:
emails = pd.read_csv('emails2.csv')

In [6]:
ms = [email.message_from_string(m)["To"] for m in emails["message"][0:100]]

In [345]:
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
file       517401 non-null object
message    517401 non-null object
dtypes: object(2)
memory usage: 7.9+ MB


In [7]:
ids = {}
table = []
I = 0
for m in emails["message"]:
    obj = email.message_from_string(m)
    to = obj["To"]
    fro = obj["From"]
    try:
        if to != None:
            es = re.split('[ \r\n\t]*,[ \r\n\t]*', to)
            eg = re.split('[ \r\n\t]*,[ \r\n\t]*', fro)
            ns = re.split(',', obj["X-To"])
            ng = re.split(',', obj["X-From"])
            for i in range(len(es)):
                e = es[i]
                if not e in ids:
                    I+=1
                    ids[e] = I
                    table.append( (I, e, ns[i]) )
            for i in range(len(eg)):
                e = eg[i]
                if not e in ids:
                    I+=1
                    ids[e] = I
                    table.append( (I, e, ng[i]) )
    except TypeError:
        continue

{'tim.belden@enron.com': 1,
 'phillip.allen@enron.com': 2,
 'john.lavorato@enron.com': 3,
 'leah.arsdall@enron.com': 4,
 'randall.gay@enron.com': 5,
 'greg.piper@enron.com': 6,
 'david.l.johnson@enron.com': 7,
 'john.shafer@enron.com': 8,
 'joyce.teixeira@enron.com': 9,
 'mark.scott@enron.com': 10,
 'zimam@enron.com': 11,
 'buck.buckner@honeywell.com': 12,
 'stagecoachmama@hotmail.com': 13,
 'keith.holst@enron.com': 14,
 'david.delainey@enron.com': 15,
 'paula.harris@enron.com': 16,
 'ina.rangel@enron.com': 17,
 'tim.heizenrader@enron.com': 18,
 'pallen70@hotmail.com': 19,
 'bs_stone@yahoo.com': 20,
 'stouchstone@natsource.com': 21,
 'lkuch@mh.com': 22,
 'jeffrey.hodge@enron.com': 23,
 'kholst@enron.com': 24,
 'rlehmann@yahoo.com': 25,
 'cindy.cicchetti@enron.com': 26,
 'jsmith@austintx.com': 27,
 'christopher.calger@enron.com': 28,
 'muller@thedoghousemail.com': 29,
 'kathy.moore@enron.com': 30,
 'james.steffes@enron.com': 31,
 'cbpres@austin.rr.com': 32,
 'paul.lucci@enron.com': 33,


In [11]:
names_df = pd.DataFrame(table)

In [44]:
names_df.columns = ['id', 'e-mail', 'name']

In [18]:
names_df[:10]

,id,e-mail,name
0,1,tim.belden@enron.com,Tim Belden <Tim Belden/Enron@EnronXGate>
1,2,phillip.allen@enron.com,Phillip K Allen
2,3,john.lavorato@enron.com,John J Lavorato <John J Lavorato/ENRON@enronXg...
3,4,leah.arsdall@enron.com,Leah Van Arsdall
4,5,randall.gay@enron.com,Randall L Gay
5,6,greg.piper@enron.com,Greg Piper
6,7,david.l.johnson@enron.com,david.l.johnson@enron.com
7,8,john.shafer@enron.com,John Shafer
8,9,joyce.teixeira@enron.com,Joyce Teixeira
9,10,mark.scott@enron.com,Mark Scott


Получили таблицу с именами и эмейлами. Теперь - с письмами. (нужны дата, айди отправителя и айди получателя, текст)

In [51]:
#ids = {}
letters = []
I = 0
for m in emails["message"]:
    obj = email.message_from_string(m)
    Date = obj["Date"]
    id_get = obj["To"]
    id_sent = obj["From"]
    payload = ""
    try:
        if obj.is_multipart: 
            payload = "".join([pl for pl in obj.get_payload()])
        else:
            payload = obj.get_payload();
        to_ids = [ids[e] for e in re.split('[ \r\n\t]*,[ \r\n\t]*' , obj["To"])]
        letters.append((payload, Date, ids[obj["From"]], to_ids))
    except (TypeError, KeyError):
        continue

In [50]:
#letters

In [52]:
letters_df = pd.DataFrame(letters)

In [53]:
letters_df.columns = ['text', 'Date','from-id', 'to-id']

In [58]:
letters_df[:50]

,text,Date,from-id,to-id
0,Here is our forecast\n\n,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",2,[1]
1,Traveling to have a business meeting takes the...,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",2,[3]
2,test successful. way to go!!!,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",2,[4]
3,"Randy,\n\n Can you send me a schedule of the s...","Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",2,[5]
4,Let's shoot for Tuesday at 11:45.,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",2,[6]
5,"Greg,\n\n How about either next Tuesday or Thu...","Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",2,[6]
6,Please cc the following distribution list with...,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",2,"[7, 8]"
7,any morning between 10 and 11:30,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",2,[9]
8,1. login: pallen pw: ke9davis\n\n I don't thi...,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",2,[10]
9,---------------------- Forwarded by Phillip K ...,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",2,[11]


In [103]:
letters_info = letters_df.groupby('from-id')['text'].nunique()

In [108]:
letters_info.shape

(19750,)

Задание 2 (15 баллов)

Сформируйте матрицу $A = (a_{ij})_{i,j = 1}^N$, где $a_{ij}$ - количество писем, которое индивид $i$ отправил индивиду $j$, $N$ - количество индивидуумов

Далее из матрицы $A$ получим симметричную матрицу, которая будет показывать нам частоту коммуникаций между каждой парой индивидов: $A+A^T$. Постройте по данной матрице не направленный граф, где узлы должны называться по именам, которые вы извлекли из X-From и X-To

In [56]:
from itertools import chain

In [57]:
%matplotlib inline
import matplotlib.pyplot as plt

from itertools import combinations
from sklearn.preprocessing import LabelEncoder


In [63]:
from scipy.sparse import lil_matrix

In [95]:
A = lil_matrix((2,2))

In [96]:
A[0][0] = 0
A[0][0] += 1

NotImplementedError: adding a nonzero scalar to a sparse matrix is not supported

In [89]:
N = names_df.shape[0]
A = lil_matrix((N, N))
for letter in letters[0:10]:
    for to in letter[3]:
        try:
            A[letter[2]-1][to-1]+=1
        except:
            print(A[letter[2]-1, to-1])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [66]:
for people_id in letters:
    for i, j in combinations(people_id, 2):
        A[i, j] += 1
        A[j, i] += 1


plt.matshow(A)
plt.colorbar()

plt.xticks(np.arange(N), names_df.shape[0], rotation=90)
plt.yticks(np.arange(N), names_df.shape[0])

plt.show()

ValueError: invalid literal for int() with base 10: 't'